In [1]:
! pip install datasets
! pip install sentence-transformers
! pip install huggingface-hub
! pip install faiss-cpu


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


^C



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#for CLIP embedding search
import torch
from packaging import version
import numpy as np
from abc import ABC, abstractmethod

from transformers import CLIPModel, AutoProcessor
from sentence_transformers import SentenceTransformer
#for dataset download
import datasets
from datasets import load_dataset
from huggingface_hub import hf_hub_download
import pandas as pd

Import Dataset:

In [ ]:
dataset = load_dataset("recastai/LAION-art-EN-improved-captions", split="train")

Image to dataset


In [4]:
pip install img2dataset

   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.8 MB 640.0 kB/s eta 0:00:39
    --------------------------------------- 0.5/24.8 MB 7.5 MB/s eta 0:00:04
   --- ------------------------------------ 1.9/24.8 MB 17.0 MB/s eta 0:00:02
   ----- ---------------------------------- 3.5/24.8 MB 22.6 MB/s eta 0:00:01
   --------- ------------------------------ 5.6/24.8 MB 27.6 MB/s eta 0:00:01
   ------------ --------------------------- 7.6/24.8 MB 30.3 MB/s eta 0:00:01
   --------------- ------------------------ 9.8/24.8 MB 33.0 MB/s eta 0:00:01
   ------------------- -------------------- 12.3/24.8 MB 46.7 MB/s eta 0:00:01
   ------------------------ --------------- 15.3/24.8 MB 50.4 MB/s eta 0:00:01
   ----------------------------- ---------- 18.3/24.8 MB 59.5 MB/s eta 0:00:01
   ---------------------------------- ----- 21.2/24.8 MB 65.6 MB/s eta 0:00:01
   -------------------------------------- - 24.1/24.8 MB 65.6 MB/s e

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
clip-retrieval 2.44.0 requires pyarrow<15,>=6.0.1, but you have pyarrow 15.0.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from img2dataset import download
import shutil
import os
import pyarrow as pa
from datasets import load_dataset

if __name__ == "__main__":
    # Load your dataset
    dataset = load_dataset("recastai/LAION-art-EN-improved-captions", split="train")
    
    # Sample 15000 random entries
    sampled_dataset = dataset.shuffle(seed=0).select(range(21000))
    
    # Create a PyArrow table
    table = pa.Table.from_pydict({
        "url": sampled_dataset["url"],
        "key": sampled_dataset["key"],
        "generated_caption": sampled_dataset["generated_caption"],
        "orig_caption": sampled_dataset["orig_caption"]
    })

    # Save as Parquet file
    output_dir = os.path.abspath("img_dataset")
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
   
    parquet_path = os.path.join(output_dir, "sampled_data.parquet")
    pa.parquet.write_table(table, parquet_path)

    # Print the contents of the Parquet file for verification
    print("Contents of the Parquet file:")
    print(pa.parquet.read_table(parquet_path))


    # # Now use img2dataset with ray distributor
    # download(
    #     processes_count=16,
    #     thread_count=32,
    #     url_list=parquet_path,
    #     image_size=256,
    #     output_folder=output_dir,
    #     output_format="files",
    #     input_format="parquet",
    #     url_col="url",
    #     caption_col="generated_caption",
    #     enable_wandb=True,
    #     number_sample_per_shard=1000,
    #     distributor="ray",
    # )

Contents of the Parquet file:
pyarrow.Table
url: string
key: string
generated_caption: string
orig_caption: string
----
url: [["https://images2.bonhams.com/image?src=Images/live/2007-06/01/7445262-3-5.jpg&width=960","https://www.copia-di-arte.com/kunst/gerard_dou/stillleben_mit_knabe_die_seife_lo.jpg","https://assets.bluethumb.com.au/media/image/thumbnail/5000/415/eyJpZCI6InVwbG9hZHMvbGlzdGluZy80NzU4NDAvZ2FicmllbGxlLXRpdG8tY2FuZHktYmx1ZXRodW1iLWMyMWMuanBlZyIsInN0b3JhZ2UiOiJzdG9yZSIsIm1ldGFkYXRhIjp7ImZpbGVuYW1lIjoiZ2FicmllbGxlLXRpdG8tY2FuZHktYmx1ZXRodW1iLWMyMWMuanBlZyIsIm1pbWVfdHlwZSI6bnVsbH19?signature=17b2ab6961c33978efb62b5bbd86199ea85274e74f3f02e81a50f1d8641bcfaf","https://i.pinimg.com/236x/0e/db/f6/0edbf6d8c7241b60a28f264b7c4cc7f7--clipart.jpg","https://i.pinimg.com/736x/34/b7/be/34b7bebf35ab9e797ca1f7210a44492a--quilling-ideas-dollhouse-miniatures.jpg",...,"https://a0.muscache.com/im/pictures/b5348a75-f1d7-40f7-8c31-2c1d2c85d161.jpg?im_w=720","https://www.dashingtweeds.co.uk/wp-co

In [22]:
print(pa.parquet.read_table(parquet_path))

pyarrow.Table
url: string
key: string
generated_caption: string
orig_caption: string
----
url: [["https://images2.bonhams.com/image?src=Images/live/2007-06/01/7445262-3-5.jpg&width=960","https://www.copia-di-arte.com/kunst/gerard_dou/stillleben_mit_knabe_die_seife_lo.jpg","https://assets.bluethumb.com.au/media/image/thumbnail/5000/415/eyJpZCI6InVwbG9hZHMvbGlzdGluZy80NzU4NDAvZ2FicmllbGxlLXRpdG8tY2FuZHktYmx1ZXRodW1iLWMyMWMuanBlZyIsInN0b3JhZ2UiOiJzdG9yZSIsIm1ldGFkYXRhIjp7ImZpbGVuYW1lIjoiZ2FicmllbGxlLXRpdG8tY2FuZHktYmx1ZXRodW1iLWMyMWMuanBlZyIsIm1pbWVfdHlwZSI6bnVsbH19?signature=17b2ab6961c33978efb62b5bbd86199ea85274e74f3f02e81a50f1d8641bcfaf","https://i.pinimg.com/236x/0e/db/f6/0edbf6d8c7241b60a28f264b7c4cc7f7--clipart.jpg","https://i.pinimg.com/736x/34/b7/be/34b7bebf35ab9e797ca1f7210a44492a--quilling-ideas-dollhouse-miniatures.jpg",...,"https://a0.muscache.com/im/pictures/b5348a75-f1d7-40f7-8c31-2c1d2c85d161.jpg?im_w=720","https://www.dashingtweeds.co.uk/wp-content/uploads/2018/06/shetland

In [21]:
# Now use img2dataset function to download the images and captions from the dataset.
#I want the caption column to include both caption_col="generated_caption", and caption_col="orig_caption".


download(
        processes_count=16,
        thread_count=64,
        url_list=parquet_path,
        image_size=256,
        output_folder=output_dir,
        output_format="files",
        input_format="parquet",
        url_col="url",
        caption_col="orig_caption",
        enable_wandb=True,
        number_sample_per_shard=3000,
        distributor="multiprocessing",
    )

Starting the downloading of this file
Sharding file number 1 of 1 called e:/Projects/summer ai/huggingfacedatasets/img_dataset/sampled_data.parquet


0it [00:00, ?it/s]

File sharded in 7 shards


7it [03:48, 32.67s/it] 
